In [2]:
from dotenv import load_dotenv
from IPython.display import Markdown, display
load_dotenv()  # loads .env into this process

import os, pandas as pd, sqlalchemy as sa, psycopg

# Build engine 
engine = sa.create_engine(
    f"postgresql+psycopg://{os.getenv('PGUSER')}:{os.getenv('PGPASSWORD')}@"
    f"{os.getenv('PGHOST')}:{os.getenv('PGPORT')}/{os.getenv('PGDATABASE')}"
)

In [3]:
# Helper function to make queries easier 
def q(sql):
    return pd.read_sql(sql, engine)

In [4]:
# Row counts + distinct IDs
q("""
SELECT
    COUNT(*) AS rows,
    COUNT(DISTINCT source_id) AS distinct_ids
FROM raw.chicago_crimes;
""")

,rows,distinct_ids
0,5116,5116


In [5]:
# Look at crime types inside JSON
q("""
SELECT payload ->> 'primary_type' AS crime_type, COUNT(*) AS n
FROM raw.chicago_crimes
GROUP BY 1
ORDER BY n DESC
LIMIT 25;
""")

,crime_type,n
0,THEFT,1064
1,BATTERY,1014
2,CRIMINAL DAMAGE,524
3,ASSAULT,469
4,MOTOR VEHICLE THEFT,375
5,OTHER OFFENSE,349
6,DECEPTIVE PRACTICE,299
7,BURGLARY,225
8,NARCOTICS,162
9,ROBBERY,142


In [6]:
q("""
SELECT primary_type, COUNT(*) AS n
FROM public_stg.stg_chicago_crimes
GROUP BY 1
ORDER BY n DESC
LIMIT 25;
""")

,primary_type,n
0,THEFT,1064
1,BATTERY,1014
2,CRIMINAL DAMAGE,524
3,ASSAULT,469
4,MOTOR VEHICLE THEFT,375
5,OTHER OFFENSE,349
6,DECEPTIVE PRACTICE,299
7,BURGLARY,225
8,NARCOTICS,162
9,ROBBERY,142


In [7]:
# Time coverage 
q("""
SELECT 
    MIN(date) AS first_record,
    MAX(date) AS last_record,
    COUNT(*) AS total_rows
FROM public_stg.stg_chicago_crimes;
""")

,first_record,last_record,total_rows
0,2001-08-26 11:45:00,2025-09-04,5116


In [8]:
# Monthly trends by crime type 
q("""
SELECT 
    DATE_TRUNC('month', date) AS month,
    primary_type,
    COUNT(*) AS n
FROM public_stg.stg_chicago_crimes
GROUP BY 1,2
ORDER BY month, n DESC;
""")


,month,primary_type,n
0,2001-08-01,CRIMINAL DAMAGE,1
1,2001-08-01,CRIMINAL SEXUAL ASSAULT,1
2,2001-08-01,CRIMINAL TRESPASS,1
3,2001-11-01,CRIMINAL SEXUAL ASSAULT,1
4,2004-03-01,DECEPTIVE PRACTICE,1
...,...,...,...
183,2025-09-01,ARSON,2
184,2025-09-01,HOMICIDE,2
185,2025-09-01,PROSTITUTION,1
186,2025-09-01,OBSCENITY,1


In [9]:
# Arrest rates 
q("""
SELECT 
  primary_type,
  COUNT(*) AS total,
  SUM(CASE WHEN arrest THEN 1 ELSE 0 END) AS arrests,
  ROUND(100.0 * SUM(CASE WHEN arrest THEN 1 ELSE 0 END) / COUNT(*), 2) AS arrest_rate_pct
FROM public_stg.stg_chicago_crimes
GROUP BY primary_type
ORDER BY total DESC
LIMIT 15;
""")
  

,primary_type,total,arrests,arrest_rate_pct
0,THEFT,1064,94,8.83
1,BATTERY,1014,162,15.98
2,CRIMINAL DAMAGE,524,18,3.44
3,ASSAULT,469,60,12.79
4,MOTOR VEHICLE THEFT,375,7,1.87
5,OTHER OFFENSE,349,58,16.62
6,DECEPTIVE PRACTICE,299,15,5.02
7,BURGLARY,225,11,4.89
8,NARCOTICS,162,152,93.83
9,ROBBERY,142,13,9.15


In [10]:
# Location hot spots 
q("""
SELECT
    location_description,
    COUNT(*) AS n
FROM public_stg.stg_chicago_crimes
GROUP BY location_description
ORDER BY n DESC
LIMIT 15;
""")

,location_description,n
0,STREET,1329
1,APARTMENT,960
2,RESIDENCE,627
3,SIDEWALK,313
4,PARKING LOT / GARAGE (NON RESIDENTIAL),200
5,SMALL RETAIL STORE,161
6,ALLEY,134
7,DEPARTMENT STORE,113
8,VEHICLE NON-COMMERCIAL,92
9,RESTAURANT,87


In [ ]:
	•	notebooks/eda: add date coverage + row count checks
	•	notebooks/eda: add monthly trend analysis by crime type
	•	notebooks/eda: add arrest rate by crime type
	•	notebooks/eda: add location hotspot analysis